### Import the relevant packages

In [66]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

### Data

In [67]:
mnist_dataset, mnist_info = tfds.load('mnist', with_info=True, as_supervised=True)

In [68]:
# train, validation, test
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# We would like to scale our data in some way, to make the result more numerically stable (inputs between 0 and 1)
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

In [69]:
# Shuffle and batch the data
BUFFER_SIZE = 10000 # we can not shuffle all the data at once, when we have a lot of data.

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE) # we shuffle 10000 at a time

# after shuffling, we can finally stract the data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# Mini batch gradient descent to train our model
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE) # how many samples to take in each batch

# here we don't backpropagate, we don't need to batch, but our model expects the data in batch format too.
validation_data = validation_data.batch(num_validation_samples) 
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [70]:
# Outline the model

input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),

                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),# hidden layers
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),

                            tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


In [71]:
# Choose the optimizer and the loss function

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [72]:
# Training

NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 5s - loss: 0.4167 - accuracy: 0.8821 - val_loss: 0.2103 - val_accuracy: 0.9412
Epoch 2/5
540/540 - 2s - loss: 0.1671 - accuracy: 0.9506 - val_loss: 0.1485 - val_accuracy: 0.9598
Epoch 3/5
540/540 - 2s - loss: 0.1274 - accuracy: 0.9623 - val_loss: 0.1332 - val_accuracy: 0.9623
Epoch 4/5
540/540 - 2s - loss: 0.1056 - accuracy: 0.9677 - val_loss: 0.1081 - val_accuracy: 0.9690
Epoch 5/5
540/540 - 3s - loss: 0.0885 - accuracy: 0.9726 - val_loss: 0.0961 - val_accuracy: 0.9722


In [73]:
# Testing
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 845ms/step - loss: 0.1019 - accuracy: 0.9686


In [74]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100)) # If the accuracy of the test, is close to the validation accuracy, we know the model is well trained, and not overfitted

Test loss: 0.10. Test accuracy: 96.86%
